# fast.ai

In [ ]:
from fastai.text.all import *
path = untar_data(URLs.IMDB)

In [ ]:
dls = TextDataLoaders.from_folder(path, valid='test')

In [ ]:
# take a quick glimpse at our data
dls.show_batch()

,text,category
0,"xxbos xxmaj match 1 : xxmaj tag xxmaj team xxmaj table xxmaj match xxmaj bubba xxmaj ray and xxmaj spike xxmaj dudley vs xxmaj eddie xxmaj guerrero and xxmaj chris xxmaj benoit xxmaj bubba xxmaj ray and xxmaj spike xxmaj dudley started things off with a xxmaj tag xxmaj team xxmaj table xxmaj match against xxmaj eddie xxmaj guerrero and xxmaj chris xxmaj benoit . xxmaj according to the rules of the match , both opponents have to go through tables in order to get the win . xxmaj benoit and xxmaj guerrero heated up early on by taking turns hammering first xxmaj spike and then xxmaj bubba xxmaj ray . a xxmaj german xxunk by xxmaj benoit to xxmaj bubba took the wind out of the xxmaj dudley brother . xxmaj spike tried to help his brother , but the referee restrained him while xxmaj benoit and xxmaj guerrero",pos
1,"xxbos * ! ! - xxup spoilers - ! ! * \n\n xxmaj before i begin this , let me say that i have had both the advantages of seeing this movie on the big screen and of having seen the "" authorized xxmaj version "" of this movie , remade by xxmaj stephen xxmaj king , himself , in 1997 . \n\n xxmaj both advantages made me appreciate this version of "" the xxmaj shining , "" all the more . \n\n xxmaj also , let me say that xxmaj i 've read xxmaj mr . xxmaj king 's book , "" the xxmaj shining "" on many occasions over the years , and while i love the book and am a huge fan of his work , xxmaj stanley xxmaj kubrick 's retelling of this story is far more compelling … and xxup scary . \n\n xxmaj kubrick",pos
2,"xxbos xxmaj some have praised _ xxunk _ as a xxmaj disney adventure for adults . i do n't think so -- at least not for thinking adults . \n\n xxmaj this script suggests a beginning as a live - action movie , that struck someone as the type of crap you can not sell to adults anymore . xxmaj the "" crack staff "" of many older adventure movies has been done well before , ( think _ the xxmaj dirty xxmaj dozen _ ) but _ atlantis _ represents one of the worse films in that motif . xxmaj the characters are weak . xxmaj even the background that each member trots out seems stock and awkward at best . xxmaj an xxup md / xxmaj medicine xxmaj man , a tomboy mechanic whose father always wanted sons , if we have not at least seen these before",neg
3,"xxbos xxrep 3 * xxmaj warning - this review contains "" plot spoilers , "" though nothing could "" spoil "" this movie any more than it already is . xxmaj it really xxup is that bad . xxrep 3 * \n\n xxmaj before i begin , xxmaj i 'd like to let everyone know that this definitely is one of those so - incredibly - bad - that - you - fall - over - laughing movies . xxmaj if you 're in a lighthearted mood and need a very hearty laugh , this is the movie for you . xxmaj now without further ado , my review : \n\n xxmaj this movie was found in a bargain bin at wal - mart . xxmaj that should be the first clue as to how good of a movie it is . xxmaj secondly , it stars the lame action",neg
4,"xxbos 8 xxmaj simple xxmaj rules for xxmaj dating xxmaj my xxmaj teenage xxmaj daughter had an auspicious start . xxmaj the supremely - talented xxmaj tom xxmaj shadyac was involved in the project . xxmaj this meant that the comedy would be nothing less of spectacular , and that 's exactly what happened : the show remains one of the freshest , funniest , wittiest shows made in a very long time . xxmaj every line , facial expression , casting choice , scene , all wreaked of perfection . xxmaj there was not one episode after which i thought , "" man that was n't as good as the rest "" . xxmaj each one was a standout . xxmaj again , this is the kind of perfectionism that we 've come to expect from xxmaj tom . xxmaj for those who do n't know , xxmaj tom",pos
5,"xxbos "" the xxmaj blob "" qualifies as a cult sci - fi film not only because it launched 27 - year old xxmaj steve mcqueen on a trajectory to superstardom , but also because it exploited the popular themes both of alien invasion and teenage d

In [ ]:
# Use AWD_LSTM for now, since it is easier and faster to train than transformer
# "drop_mult" controls the magnitude of all dropouts in the model
learn = text_classifier_learner(dls, AWD_LSTM, drop_mult=0.5, metrics=accuracy)

/usr/local/lib/python3.11/dist-packages/fastai/text/learner.py:149: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  wgts = torch.load(wgts_fname, map_location = lambda storage

In [ ]:
learn.fine_tune(4, 1e-2)

epoch,train_loss,valid_loss,accuracy,time
0,0.456952,0.382421,0.828480,03:21


epoch,train_loss,valid_loss,accuracy,time
0,0.312554,0.252524,0.897520,07:01
1,0.247920,0.198694,0.922560,07:02
2,0.175353,0.195914,0.925880,07:02
3,0.142009,0.196380,0.928160,07:01


In [ ]:
learn.show_results()

,text,category,category_
0,"xxbos xxmaj there 's a sign on xxmaj the xxmaj lost xxmaj highway that says : \n\n * major xxup spoilers xxup ahead * \n\n ( but you already knew that , did n't you ? ) \n\n xxmaj since there 's a great deal of people that apparently did not get the point of this movie , xxmaj i 'd like to contribute my interpretation of why the plot makes perfect sense . xxmaj as others have pointed out , one single viewing of this movie is not sufficient . xxmaj if you have the xxup dvd of xxup md , you can "" cheat "" by looking at xxmaj david xxmaj lynch 's "" top 10 xxmaj hints to xxmaj unlocking xxup md "" ( but only upon second or third viewing , please . ) ;) \n\n xxmaj first of all , xxmaj mulholland xxmaj drive is",pos,pos
1,"xxbos xxmaj possible xxmaj spoilers xxmaj as xxmaj peter xxmaj jackson has so brilliantly shown in his ` lord of the xxmaj rings ' trilogy , it is quite possible to make good films , even great ones , within the heroic fantasy genre . xxmaj the genre has distinguished literary antecedents , dating back to the mediaeval chivalric romances , particularly the xxmaj arthurian legends , upon which xxmaj tolkien and other authors have drawn . ` lord of the xxmaj rings ' apart , however , it is difficult to think of any other sword - and - sorcery films which are any good . i would agree with the reviewer who said that the best of a bad bunch was xxmaj ron xxmaj howard 's ` willow ' , and even that achieved little distinction other than that of being a merely mediocre film as opposed to",neg,neg
2,"xxbos "" empire xxmaj strikes xxmaj back "" director xxmaj irvin xxmaj kershner 's "" never xxmaj say xxmaj never xxmaj again , "" a remake of the 1965 xxmaj james xxmaj bond movie "" thunderball , "" does n't surpasses the xxmaj terence xxmaj young original , but this non - harry xxmaj xxunk & xxmaj albert xxup r. xxmaj broccoli film is well worth watching if you call yourself a 007 aficionado . xxmaj nevertheless , despite its shortage of clever gadgets and the lack of a vibrant musical score , "" never xxmaj say xxmaj never xxmaj again "" rates as an above - average , suspenseful doomsday thriller with top - flight performances by a seasoned cast including xxmaj sean xxmaj connery , xxmaj kim xxmaj basinger , xxmaj klaus xxmaj maria xxmaj brandauer , xxmaj max xxmaj von xxmaj sydow , xxmaj barbara xxmaj carrera",pos,pos
3,"xxbos xxmaj during an eight - year stretch of the 1980s when xxmaj charles xxmaj bronson made nine films , only one was released by a company other than the xxmaj cannon xxmaj group : ' the xxmaj evil xxmaj that xxmaj men xxmaj do , ' a tristar xxmaj films pickup from xxmaj sir xxmaj lew xxmaj grade 's xxup itc xxmaj entertainment firm . xxmaj bronson was already in the thick of his collaborations with director xxup j. xxmaj lee xxmaj thompson , which ran through numerous actioners until ' kinjite : xxmaj forbidden xxmaj subjects ' in 1989 . \n\n xxmaj expectations should run pretty high with xxmaj bronson and xxmaj thompson working for a better - funded outfit like xxup itc , but ' the xxmaj evil xxmaj that xxmaj man xxmaj do ' is a great disappointment on many levels . xxmaj while still from",neg,neg
4,"xxbos xxmaj no one is going to mistake xxup the xxup squall for a good movie , but it sure is a memorable one . xxmaj once you 've taken in xxmaj myrna xxmaj loy 's performance as xxmaj nubi the hot - blooded gypsy girl you 're not likely to forget the experience . xxmaj when this film was made the exotically beautiful xxmaj miss xxmaj loy was still being cast as foreign vixens , often xxmaj asian and usually sinister . xxmaj she 's certainly an eyeful here . xxmaj it appears that her skin was darkened and her hair was curled . xxmaj in most scenes she 's barefoot and wearing little more than a skirt and a loose - fitting peasant blouse , while in one scene she wears nothing but a patterned towel . i suppose xxmaj i 'm focusing on xxmaj miss xxmaj loy",neg,neg
5,"xxbos xxmaj

In [ ]:
learn.predict("That movie was wicked cool!")

('pos', tensor(1), tensor([0.0928, 0.9072]))

## Using the method ULMFiT

In [ ]:
# "valid_pct" makes a random split is performed (with an optional seed) by setting aside that percentage of the data for the validation set
dls_lm = TextDataLoaders.from_folder(path, is_lm=True, valid_pct=0.1)
dls_lm.show_batch(max_n=5)

,text,text_
0,"xxbos xxmaj for anyone who 's judged others at first meeting , here is the perfect tutorial on depth of character . xxmaj the grumpy old lady has a soft , thoughtful heart - and needs new friends . xxmaj the flighty , unsure , ' ditsy ' dame who makes inappropriate , uncomfortable comments - sees deep into your soul and has pure love for all . xxmaj the cold ,","xxmaj for anyone who 's judged others at first meeting , here is the perfect tutorial on depth of character . xxmaj the grumpy old lady has a soft , thoughtful heart - and needs new friends . xxmaj the flighty , unsure , ' ditsy ' dame who makes inappropriate , uncomfortable comments - sees deep into your soul and has pure love for all . xxmaj the cold , prim"
1,"'s xxup apes doing the chasing - or at least , it * would * have been a surprise if no one saw xxmaj planet xxmaj of xxmaj the xxmaj apes xxup thirty - three xxup years xxup ago . \n\n xxmaj since xxmaj marky xxmaj mark did not get to show his pecs , take down his pants , or bust his lame whiteboy rap , he was characterless . xxmaj","xxup apes doing the chasing - or at least , it * would * have been a surprise if no one saw xxmaj planet xxmaj of xxmaj the xxmaj apes xxup thirty - three xxup years xxup ago . \n\n xxmaj since xxmaj marky xxmaj mark did not get to show his pecs , take down his pants , or bust his lame whiteboy rap , he was characterless . xxmaj michael"
2,"and plans to exhibit the greatest magic trick in history ; entitled "" the xxmaj girl and the xxmaj buzz xxmaj saw "" . xxmaj gallico 's promising solo career is abruptly ruined before it even begins when his previous employer xxmaj ross xxmaj ormond appears on stage and shoves a contract under his nose , stating that all of xxmaj gallico 's inventions are the rightful property of the company .","plans to exhibit the greatest magic trick in history ; entitled "" the xxmaj girl and the xxmaj buzz xxmaj saw "" . xxmaj gallico 's promising solo career is abruptly ruined before it even begins when his previous employer xxmaj ross xxmaj ormond appears on stage and shoves a contract under his nose , stating that all of xxmaj gallico 's inventions are the rightful property of the company . xxmaj"
3,"i would n't totally ridicule every person responsible for production of this film is if i learned that it was produced entirely by 11 year old 's . \n\n xxmaj really though , even with all of the criticism i offer here , xxmaj i 'd suggest watching this movie solely based on the fact that it may very well be the worst movie ever , and because of this is quite","would n't totally ridicule every person responsible for production of this film is if i learned that it was produced entirely by 11 year old 's . \n\n xxmaj really though , even with all of the criticism i offer here , xxmaj i 'd suggest watching this movie solely based on the fact that it may very well be the worst movie ever , and because of this is quite comical"
4,sketch comedy going back at least ten years prior . xxmaj the only way i could even forgive this film even being made is if it was at gunpoint . xxmaj retro ? xxmaj hardly . xxmaj sketches about clowns subtly xxunk children may be cutting edge in some circles ( and it could actually have been funny ) but it just comes off as really quite sad . xxmaj what kept,comedy going back at least ten years prior . xxmaj the only way i could even forgive this film even being made is if it was at gunpoint . xxmaj retro ? xxmaj hardly . xxmaj sketches about clowns subtly xxunk children may be cutting edge in some circles ( and it could actually have been funny ) but it just comes off as really quite sad . xxmaj what kept me


In [ ]:
# ".to_fp16()" help speed up training on GPUs that have Tensor Cores
learn = language_model_learner(
    dls_lm, AWD_LSTM, metrics=[accuracy, Perplexity()],
    path=path, wd=0.1).to_fp16()

/usr/local/lib/python3.11/dist-packages/fastai/text/learner.py:149: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  wgts = torch.load(wgts_fname, map_location = lambda storage

In [ ]:
learn.fit_one_cycle(1, 1e-2)

/usr/local/lib/python3.11/dist-packages/fastai/callback/fp16.py:47: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  self.autocast,self.learn.scaler,self.scales = autocast(dtype=dtype),GradScaler(**self.kwargs),L()
/usr/local/lib/python3.11/dist-packages/torch/amp/autocast_mode.py:266: UserWarning: User provided device_type of 'cuda', but CUDA is not available. Disabling
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/fastai/callback/fp16.py:47: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  self.autocast,self.learn.scaler,self.scales = autocast(dtype=dtype),GradScaler(**self.kwargs),L()
/usr/local/lib/python3.11/dist-packages/torch/amp/grad_scaler.py:132: UserWarning: torch.cuda.amp.GradScaler is enabled, but CUDA is not available.  Disabling.
  warnings.warn(


epoch,train_loss,valid_loss,accuracy,perplexity,time


In [ ]:
learn.save('1epoch')

Path('/root/.fastai/data/imdb/models/1epoch.pth')

In [1]:
learn = learn.load('1epoch')

NameError: name 'learn' is not defined

In [ ]:
learn.unfreeze()
learn.fit_one_cycle(10, 1e-3)

/usr/local/lib/python3.11/dist-packages/fastai/callback/fp16.py:47: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  self.autocast,self.learn.scaler,self.scales = autocast(dtype=dtype),GradScaler(**self.kwargs),L()
/usr/local/lib/python3.11/dist-packages/torch/amp/autocast_mode.py:266: UserWarning: User provided device_type of 'cuda', but CUDA is not available. Disabling
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/fastai/callback/fp16.py:47: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  self.autocast,self.learn.scaler,self.scales = autocast(dtype=dtype),GradScaler(**self.kwargs),L()
/usr/local/lib/python3.11/dist-packages/torch/amp/grad_scaler.py:132: UserWarning: torch.cuda.amp.GradScaler is enabled, but CUDA is not available.  Disabling.
  warnings.warn(


epoch,train_loss,valid_loss,accuracy,perplexity,time


In [ ]:
# Encoder : The model not including the final layer. It maps input text to some tensors that we use to solve NLP tasks
# Decoder maps the vectors onto some task-specific data structure
learn.save_encoder('finetuned')

NameError: name 'learn' is not defined

In [ ]:
# generate random reviews
TEXT = "I liked this movie because"
N_WORDS = 40
N_SENTENCES = 2
preds = [learn.predict(TEXT, N_WORDS, temperature=0.75)
         for _ in range(N_SENTENCES)]

In [ ]:
print("\n".join(preds))

In [ ]:
dls_clas = TextDataLoaders.from_folder(
    untar_data(URLs.IMDB), valid='test',
    text_vocab=dls_lm.vocab)

In [ ]:
learn = text_classifier_learner(dls, AWD_LSTM, drop_mult=0.5, metrics=accuracy)

In [ ]:
learn = learn.load_encoder('finetuned')

In [ ]:
learn.fit_one_cycle(1, 2e-2)

In [ ]:
learn.freeze_to(-2)   # freeze all except the last two parameter groups
learn.fit_one_cycle(1, slice(1e-2/(2.6**4),1e-2))

In [ ]:
learn.freeze_to(-3)
learn.fit_one_cycle(1, slice(5e-3/(2.6**4),5e-3))

In [ ]:
learn.unfreeze()
learn.fit_one_cycle(2, slice(1e-3/(2.6**4),1e-3))

# Hugging Face

In [ ]:
import torch
from transformers import GPT2Tokenizer, GPT2LMHeadModel

# Load pretrained model tokenizer (vocabulary)
tokenizer = GPT2Tokenizer.from_pretrained('gpt2')

# Encode a text inputs
text = "With great power comes great "
indexed_tokens = tokenizer.encode(text)

# Convert indexed tokens in a PyTorch tensor
tokens_tensor = torch.tensor([indexed_tokens])

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

In [ ]:
print(tokens_tensor)

tensor([[3152, 1049, 1176, 2058, 1049,  220]])


In [ ]:
# Load pretrained model (weights)
model = GPT2LMHeadModel.from_pretrained('gpt2')

# Set the model in evaluation mode to deactivate the DropOut modules
# This is IMPORTANT to have reproducible results during evaluation!
model.eval()

# Predict all tokens
with torch.no_grad():
    outputs = model(tokens_tensor)
    predictions = outputs[0]

# Get the predicted next subword
predicted_index = torch.argmax(predictions[0, -1, :]).item()
predicted_text = tokenizer.decode(indexed_tokens + [predicted_index])
print(predicted_text)

model.safetensors:   0%|          | 0.00/548M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

With great power comes great ills


In [ ]:
print(predictions)
print(predictions[0, -1, :])
predictions.shape

tensor([[[ -32.9690,  -32.2417,  -35.0687,  ...,  -37.8426,  -38.3769,
           -32.6595],
         [ -88.2333,  -87.4561,  -94.1138,  ...,  -91.9931,  -95.2675,
           -90.7903],
         [ -62.1415,  -64.1625,  -72.3117,  ...,  -72.0955,  -71.1990,
           -66.7284],
         [ -94.2255,  -94.3511, -101.4683,  ..., -100.7294, -100.7159,
           -97.0163],
         [ -99.9127,  -99.7470, -106.7008,  ..., -106.1145, -109.8075,
          -102.8086],
         [ -57.2461,  -60.1474,  -63.3509,  ...,  -72.5493,  -69.3108,
           -61.1931]]])
tensor([-57.2461, -60.1474, -63.3509,  ..., -72.5493, -69.3108, -61.1931])


torch.Size([1, 6, 50257])

In [ ]:
model = GPT2LMHeadModel.from_pretrained('gpt2')

In [ ]:
model.eval()

GPT2LMHeadModel(
  (transformer): GPT2Model(
    (wte): Embedding(50257, 768)
    (wpe): Embedding(1024, 768)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0-11): 12 x GPT2Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2Attention(
          (c_attn): Conv1D(nf=2304, nx=768)
          (c_proj): Conv1D(nf=768, nx=768)
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (mlp): GPT2MLP(
          (c_fc): Conv1D(nf=3072, nx=768)
          (c_proj): Conv1D(nf=768, nx=3072)
          (act): NewGELUActivation()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
    )
    (ln_f): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
  )
  (lm_head): Linear(in_features=768, out_features=50257, bias=False)
)

In [ ]:
with torch.no_grad():
    outputs = model(tokens_tensor)
    predictions = outputs[0]    # a 'torch.tensor' with values that describe the probability of each word

In [ ]:
len(outputs)
outputs[0].shape

torch.Size([1, 6, 50257])

In [ ]:
# perform argmax in first text, last word, all probabilities
predicted_index = torch.argmax(predictions[0, -1, :]).item()

In [ ]:
predicted_text = tokenizer.decode(indexed_tokens + [predicted_index])
print(predicted_text)

With great power comes great ills
